<a href="https://colab.research.google.com/github/dhairyachandra/Module-1-Deep-Learning-ICP/blob/master/Federated_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Section: Federated Learning

# Lesson: Introducing Federated Learning

Federated Learning is a technique for training Deep Learning models on data to which you do not have access. Basically:

Federated Learning: Instead of bringing all the data to one machine and training a model, we bring the model to the data, train it locally, and merely upload "model updates" to a central server.

Use Cases:

    - app company (Texting prediction app)
    - predictive maintenance (automobiles / industrial engines)
    - wearable medical devices
    - ad blockers / autotomplete in browsers (Firefox/Brave)
    
Challenge Description: data is distributed amongst sources but we cannot aggregated it because of:

    - privacy concerns: legal, user discomfort, competitive dynamics
    - engineering: the bandwidth/storage requirements of aggregating the larger dataset

# Lesson: Introducing / Installing PySyft

In order to perform Federated Learning, we need to be able to use Deep Learning techniques on remote machines. This will require a new set of tools. Specifically, we will use an extensin of PyTorch called PySyft.

### Install PySyft

- If you are using Google Colab, you can simply install PySyft using the following command:
`! pip install syft`

- If you are using PySyft locally, the easiest way to install the required libraries is with [Conda](https://docs.conda.io/projects/conda/en/latest/user-guide/overview.html). Create a new environment, then install the dependencies in that environment. In your terminal:

```bash
conda create -n pysyft python=3
conda activate pysyft # some older version of conda require "source activate pysyft" instead.
conda install jupyter notebook
pip install syft
pip install numpy
```

If you have any errors relating to zstd - run the following (if everything above installed fine then skip this step):

```
pip install --upgrade --force-reinstall zstd
```

and then retry installing syft (pip install syft).

If you are using Windows, I suggest installing [Anaconda and using the Anaconda Prompt](https://docs.anaconda.com/anaconda/user-guide/getting-started/) to work from the command line. 

With this environment activated and in the repo directory, launch Jupyter Notebook:

```bash
jupyter notebook
```

and re-open this notebook on the new Jupyter server.



In [0]:
import syft

In [126]:
! pip install syft

In [0]:
import torch as th
import syft as sy

In [129]:
hook = sy.TorchHook(th)  # this line of code creates an instance of th with its backend APIs modified with Syft functions

In [130]:
x = th.tensor([1,2,3,4,5]) # notice that Torch functionalities still behave the same
x

tensor([1, 2, 3, 4, 5])

# Lesson: Basic Remote Execution in PySyft

## PySyft => Remote PyTorch

The essence of Federated Learning is the ability to train models in parallel on a wide number of machines. Thus, we need the ability to tell remote machines to execute the operations required for Deep Learning.

Thus, instead of using Torch tensors - we're now going to work with **pointers** to tensors. Let me show you what I mean. First, let's create a "pretend" machine owned by a "pretend" person - we'll call him Bob.

In [0]:
bob = sy.VirtualWorker(hook, id="bob") # creates a virtual worker (a simulaion to interface to Bob machines)

In [132]:
print(f'Type: {type(bob._objects)} \nValue: {bob._objects}')

Type: <class 'dict'> 
Value: {40833737911: tensor([1, 2, 3, 4, 5]), 63696882725: tensor([[1., 1.],
        [0., 1.]], requires_grad=True), 99566913073: tensor([[1.],
        [1.]], requires_grad=True), 51708343606: Parameter containing:
tensor([[0.1073, 0.6382]], requires_grad=True), 13375451655: Parameter containing:
tensor([0.6841], requires_grad=True)}


In [0]:
x = th.tensor([1,2,3,4,5])

In [0]:
x =  x.send(bob) # send this data to bob

In [135]:
bob._objects

{13375451655: Parameter containing:
 tensor([0.6841], requires_grad=True),
 40479844113: tensor([1, 2, 3, 4, 5]),
 40833737911: tensor([1, 2, 3, 4, 5]),
 51708343606: Parameter containing:
 tensor([[0.1073, 0.6382]], requires_grad=True),
 63696882725: tensor([[1., 1.],
         [0., 1.]], requires_grad=True),
 99566913073: tensor([[1.],
         [1.]], requires_grad=True)}

In [136]:
# What's the type of the pointer? and wht's the reason behind this type?
# What's its value?

print(f'Type: {type(x)} \nValue: {x}')

Type: <class 'torch.Tensor'> 
Value: (Wrapper)>[PointerTensor | me:7627685751 -> bob:40479844113]


In [137]:
x.location # where the Tensor located?

<VirtualWorker id:bob #objects:6>

In [138]:
x.id # the pointer ID at our machine

7627685751

In [139]:
x.id_at_location # the ID of the tensor at the remote worker

40479844113

In [140]:
x.owner

<VirtualWorker id:me #objects:0>

In [141]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [142]:
x

tensor([1, 2, 3, 4, 5])

In [143]:
bob._objects

{13375451655: Parameter containing:
 tensor([0.6841], requires_grad=True),
 40833737911: tensor([1, 2, 3, 4, 5]),
 51708343606: Parameter containing:
 tensor([[0.1073, 0.6382]], requires_grad=True),
 63696882725: tensor([[1., 1.],
         [0., 1.]], requires_grad=True),
 99566913073: tensor([[1.],
         [1.]], requires_grad=True)}

# Project: Experience with Remote Tensors

In this project, I want you to .send() and .get() a tensor to TWO workers by calling .send(bob,alice). This will first require the creation of another VirtualWorker called alice.

In [0]:
# create a second Virtual worker and call it Alice
alice = sy.VirtualWorker(hook, id='alice')

In [0]:
# 1 - create some data (a tensor)
# 2- send the data to bob and alice
x = th.tensor([2,3,4])
x = x.send(bob, alice)

In [146]:
# notice what does the send fucntion on two workers return; a multi-pointer
x

(Wrapper)>[MultiPointerTensor]
	-> [PointerTensor | me:64089717240 -> bob:45733672819]
	-> [PointerTensor | me:51560960197 -> alice:83423405173]

In [147]:
# what does .child on the pointer object return?
# what does .child.child on the pointer object return? 
print(f'Type: {type(x.child)} \nValue: {x.child}')
print(f'Type: {type(x.child.child)} \nValue: {x.child.child}')

Type: <class 'syft.generic.pointers.multi_pointer.MultiPointerTensor'> 
Value: [MultiPointerTensor]
	-> [PointerTensor | me:64089717240 -> bob:45733672819]
	-> [PointerTensor | me:51560960197 -> alice:83423405173]
Type: <class 'dict'> 
Value: {'bob': [PointerTensor | me:64089717240 -> bob:45733672819], 'alice': [PointerTensor | me:51560960197 -> alice:83423405173]}


In [148]:
bob._objects

{13375451655: Parameter containing:
 tensor([0.6841], requires_grad=True),
 40833737911: tensor([1, 2, 3, 4, 5]),
 45733672819: tensor([2, 3, 4]),
 51708343606: Parameter containing:
 tensor([[0.1073, 0.6382]], requires_grad=True),
 63696882725: tensor([[1., 1.],
         [0., 1.]], requires_grad=True),
 99566913073: tensor([[1.],
         [1.]], requires_grad=True)}

In [149]:
alice._objects

{17816553461: tensor([[0.],
         [0.]], requires_grad=True), 24338575783: Parameter containing:
 tensor([0.6841], requires_grad=True), 30792796597: tensor([[1., 0.],
         [0., 0.]], requires_grad=True), 60128539608: Parameter containing:
 tensor([[0.1073, 0.6382]], requires_grad=True), 83423405173: tensor([2, 3, 4])}

In [150]:
# try the .get() on the pointer
x = x.get()
x

[tensor([2, 3, 4]), tensor([2, 3, 4])]

In [151]:
alice._objects

{17816553461: tensor([[0.],
         [0.]], requires_grad=True), 24338575783: Parameter containing:
 tensor([0.6841], requires_grad=True), 30792796597: tensor([[1., 0.],
         [0., 0.]], requires_grad=True), 60128539608: Parameter containing:
 tensor([[0.1073, 0.6382]], requires_grad=True)}

In [152]:
bob._objects

{13375451655: Parameter containing:
 tensor([0.6841], requires_grad=True),
 40833737911: tensor([1, 2, 3, 4, 5]),
 51708343606: Parameter containing:
 tensor([[0.1073, 0.6382]], requires_grad=True),
 63696882725: tensor([[1., 1.],
         [0., 1.]], requires_grad=True),
 99566913073: tensor([[1.],
         [1.]], requires_grad=True)}

In [0]:
# 1 - create some data (a tensor)
# 2- send the data to bob and alice
x = th.tensor([2, 5, 4])
x = x.send(bob, alice)

In [154]:
# try the .get(sum_results=True) on your pointer
sum = x.get(sum_results=True)
sum

tensor([ 4, 10,  8])

# Lesson: Introducing Remote Arithmetic

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(bob)

In [156]:
x # a pointer to a remote tensor located at bob

(Wrapper)>[PointerTensor | me:19362943879 -> bob:83259292590]

In [157]:
y # a pointer to another remote tensor located at bob

(Wrapper)>[PointerTensor | me:17769837398 -> bob:61601786309]

In [158]:
z = x + y # treat those tensors (i.e., x and y) as local tensors, but they are actually executed remotely
z

(Wrapper)>[PointerTensor | me:14831139631 -> bob:34827382735]

In [159]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [0]:
x = th.tensor([1.,2,3,4,5], requires_grad=True).send(bob)
y = th.tensor([1.,1,1,1,1], requires_grad=True).send(bob)

In [161]:
z = (x + y).sum()
z = z.get()
z

tensor(20., requires_grad=True)

# Project: Learn a Simple Linear Model

In this project, I'd like for you to create a simple linear model which will solve for the following dataset below. You should use only Variables and .backward() to do so (no optimizers or nn.Modules). 

Furthermore, you must do so with both the data and the model being located on Bob's machine.

In [162]:
import torch as th
import syft as sy

hook = sy.TorchHook(th)  
bob = sy.VirtualWorker(hook, id="bob")

bob

<VirtualWorker id:bob #objects:9>

In [0]:
# create some toy data for our model
input_data = th.tensor([[1., 1],[0.5, 1],[1, 0],[0, 0]], requires_grad=True).send(bob)
output_data = th.tensor([[1.],[1],[0],[0]], requires_grad=True).send(bob)

In [0]:
# create some linear weights and send them to bob
weights = th.tensor([[0.01],[0.01]], requires_grad = True).send(bob)

In [165]:
# create a linear model and train it on Bob's machine
# remember how to create a linear model? No? :( 
# Here's the main steps:
#    1- find a prediction
#    2- calcualte the loss (a mean square loss)
#    3- backpropogate using the backword() function
#    4- DO NOT forget to clear your gradients after updating the weights
#    weights.data.sub_(weights.grad * lr)

prediction = input_data.mm(weights)
prediction

(Wrapper)>[PointerTensor | me:23400541624 -> bob:12812946830]

In [166]:
lr = 0.1

for i in range(10):
  prediction = input_data.mm(weights)
  loss = ((prediction - output_data)**2).mean()
  loss.backward()

  weights.data.sub_(weights.grad * lr)
  
  weights.grad *= 0

  print(loss.get().data)

tensor(0.4827)
tensor(0.3459)
tensor(0.2529)
tensor(0.1893)
tensor(0.1455)
tensor(0.1150)
tensor(0.0935)
tensor(0.0781)
tensor(0.0668)
tensor(0.0584)


# Lesson: Garbage Collection and Common Errors


In [0]:
bob = bob.clear_objects() # clear the contents of a remote object

In [168]:
bob._objects

{}

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [170]:
bob._objects

{41682362534: tensor([1, 2, 3, 4, 5])}

In [0]:
del x  # delete the pointer to the remote object

In [172]:
bob._objects

{}

In [180]:
x.child.garbage_collect_data  # True by default

True

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [182]:
bob._objects

{9375671842: tensor([1, 2, 3, 4, 5])}

In [0]:
x = "asdf"

In [184]:
bob._objects

{}

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [186]:
x

(Wrapper)>[PointerTensor | me:85893582033 -> bob:71449412014]

In [187]:
bob._objects

{71449412014: tensor([1, 2, 3, 4, 5])}

In [0]:
x = "asdf"

In [189]:
bob._objects  # some error from Jupyter is casuing this to appear!

{71449412014: tensor([1, 2, 3, 4, 5])}

In [0]:
del x

In [191]:
bob._objects

{71449412014: tensor([1, 2, 3, 4, 5])}

In [192]:
bob = bob.clear_objects() # erase force 
bob._objects

{}

In [0]:
for i in range(1000):
    x = th.tensor([1,2,3,4,5]).send(bob)

In [194]:
bob._objects # notice that there is only a single tensor in bob

{74162512514: tensor([1, 2, 3, 4, 5])}

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(bob)

In [0]:
z = x + y

In [0]:
alice = sy.VirtualWorker(hook, id="alice")
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(alice)

In [198]:
z = x + y

TensorsNotCollocatedException: ignored

# Lesson: Toy Federated Learning

Let's start by training a toy model the centralized way. This is about a simple as models get. We first need:

- a toy dataset
- a model
- some basic training logic for training a model to fit the data.

In [0]:
from torch import nn, optim

In [0]:
# A Toy Dataset
data = th.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad=True)
target = th.tensor([[1.],[1], [0], [0]], requires_grad=True)

In [0]:
# A Toy Model
model = nn.Linear(2,1)

In [0]:
opt = optim.SGD(params=model.parameters(), lr=0.1)

In [203]:
def train(iterations=10):
    for iter in range(iterations):
        opt.zero_grad()

        pred = model(data)

        loss = ((pred - target)**2).mean()

        loss.backward()

        opt.step()

        print(loss.data)
        
train()

tensor(0.5228)
tensor(0.4676)
tensor(0.4212)
tensor(0.3810)
tensor(0.3453)
tensor(0.3133)
tensor(0.2845)
tensor(0.2585)
tensor(0.2349)
tensor(0.2136)


# nothing was federated up to this point!

In [204]:
# let's reeat the previous experiment in a FL approach:

import torch as th
import syft as sy

hook = sy.TorchHook(th)  

bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")

bob = bob.clear_objects()
alice = alice.clear_objects()

In [0]:
# create local datasets at Bob and Alice
data_bob = th.tensor([[1.,1],[0,1]], requires_grad=True).send(bob)
target_bob = th.tensor([[1.],[1]], requires_grad=True).send(bob)

data_alice = th.tensor([[1., 0],[0, 0]], requires_grad=True).send(alice)
target_alice = th.tensor([[0.],[0]], requires_grad=True).send(alice)

In [206]:
datasets = [(data_bob, target_bob), (data_alice, target_alice)]
datasets

[((Wrapper)>[PointerTensor | me:77193972027 -> bob:56916238228],
  (Wrapper)>[PointerTensor | me:23517303378 -> bob:11803985114]),
 ((Wrapper)>[PointerTensor | me:28445372817 -> alice:25153299233],
  (Wrapper)>[PointerTensor | me:39575982712 -> alice:97038213004])]

In [0]:
def train(iterations=3):

    model = nn.Linear(2,1)
    opt = optim.SGD(params=model.parameters(), lr=0.1)
    
    for iter in range(iterations):

        for _data, _target in datasets:

            # send model to the data
            model = model.send(_data.location)

            # do training on the remote machine
            # 1 zero the gradients
            opt.zero_grad()

            # 2 calculate predictions
            pred = model(data)

            # 3 calculate loss -- MSE
            loss = ((pred - target)**2).mean()

            # 4 calculate gradeints
            loss.backward()

            # 5 update weights
            opt.step()

            # return the model to the local machine
            model = model.get()
            
            
        print(loss.get())

In [0]:
train(3)

tensor(0.1480, requires_grad=True)
tensor(0.0397, requires_grad=True)
tensor(0.0210, requires_grad=True)


# Lesson: Advanced Remote Execution Tools

In the last section we trained a toy model using Federated Learning. We did this by calling .send() and .get() on our model, sending it to the location of training data, updating it, and then bringing it back. However, at the end of the example we realized that we needed to go a bit further to protect people privacy. Namely, we want to average the gradients BEFORE calling .get(). That way, we won't ever see anyone's exact gradient (thus better protecting their privacy!!!)

But, in order to do this, we need a few more pieces:

- use a pointer to send a Tensor directly to another worker

And in addition, while we're here, we're going to learn about a few more advanced tensor operations as well which will help us both with this example and a few in the future!

In [208]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [0]:
x = x.send(alice)

In [211]:
bob._objects

{58653526441: tensor([1, 2, 3, 4, 5])}

In [212]:
alice._objects

{88752471581: (Wrapper)>[PointerTensor | alice:88752471581 -> bob:58653526441]}

In [0]:
y = x + x

In [214]:
y

(Wrapper)>[PointerTensor | me:31623459391 -> alice:75089863172]

In [215]:
bob._objects

{58653526441: tensor([1, 2, 3, 4, 5]),
 84864182190: tensor([ 2,  4,  6,  8, 10])}

In [216]:
alice._objects

{38737676964: (Wrapper)>[PointerTensor | alice:75089863172 -> bob:84864182190],
 75089863172: (Wrapper)>[PointerTensor | alice:75089863172 -> bob:84864182190],
 88752471581: (Wrapper)>[PointerTensor | alice:88752471581 -> bob:58653526441]}

In [0]:
jon = sy.VirtualWorker(hook, id="jon")

In [218]:
bob.clear_objects()
alice.clear_objects()



<VirtualWorker id:alice #objects:0>

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob).send(alice)
y = th.tensor([1,2,3,4,5]).send(bob).send(jon)

In [220]:
z = x + y

TensorsNotCollocatedException: ignored

In [221]:
x = x.get() # to get the data back
x

(Wrapper)>[PointerTensor | me:17123959805 -> bob:84666669294]

In [222]:
bob._objects

{56318651985: tensor([1, 2, 3, 4, 5]), 84666669294: tensor([1, 2, 3, 4, 5])}

In [223]:
alice._objects

{}

In [224]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [225]:
bob._objects

{56318651985: tensor([1, 2, 3, 4, 5])}

# Lesson: Pointer Chain Operations

In [226]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [228]:
bob._objects

{24230173704: tensor([1, 2, 3, 4, 5])}

In [229]:
alice._objects

{}

In [230]:
x.move(alice)

(Wrapper)>[PointerTensor | me:63624864827 -> alice:76631467364]

In [231]:
x

(Wrapper)>[PointerTensor | me:63624864827 -> alice:76631467364]

In [232]:
bob._objects

{}

In [233]:
alice._objects

{76631467364: tensor([1, 2, 3, 4, 5])}

In [234]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [236]:
bob._objects

{70156915417: tensor([1, 2, 3, 4, 5])}

In [237]:
alice._objects

{23961756354: (Wrapper)>[PointerTensor | alice:23961756354 -> bob:70156915417]}

In [238]:
x

(Wrapper)>[PointerTensor | me:45210844277 -> alice:23961756354]

In [239]:
x.remote_get() # inplace operation

(Wrapper)>[PointerTensor | me:45210844277 -> alice:23961756354]

In [240]:
bob._objects

{}

In [241]:
alice._objects

{23961756354: tensor([1, 2, 3, 4, 5])}

In [242]:
x.move(bob)

(Wrapper)>[PointerTensor | me:31284003383 -> bob:45210844277]

In [243]:
x

(Wrapper)>[PointerTensor | me:31284003383 -> bob:45210844277]

In [244]:
bob._objects

{45210844277: tensor([1, 2, 3, 4, 5])}

In [245]:
alice._objects

{}


**Exercise:**

To avoid exposing gradients among participants, you need to send the gradiants to a TRUSTED thirdparty (trusted aggregator) who will aggregate the models and then send the final model to the server (local worker). In this way, we assure that none of the participating workers can access the aggregated model!

1. create a dataset for each worker (create two)
2. create a model for each worker and train it remotely on each worker
3. send those two models using the *move()* function to a third worker
4. the third workers aggregates the two models (find their mean)
5. send the aggregated model to the main server (local worker) using the *get()* function



---

To set the weights: `model.weight.set_()`



In [0]:
import syft as sy
import torch as th
from torch import nn, optim

In [0]:
# create workers
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
secureWorker = sy.VirtualWorker(hook, id="secureWorker")

In [248]:
# Make each worker aware of the other workers
bob.add_workers([alice,secureWorker])
alice.add_workers([bob,secureWorker])
secureWorker.add_workers([bob, alice])

<VirtualWorker id:secureWorker #objects:0>

In [0]:
# A Toy Dataset
data = th.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad=True)
target = th.tensor([[1.],[1], [0], [0]], requires_grad=True)

In [0]:
# create local datasets at Bob and Alice
data_bob = th.tensor([[1.,1],[0,1]], requires_grad=True).send(bob)
target_bob = th.tensor([[1.],[1]], requires_grad=True).send(bob)

data_alice = th.tensor([[1., 0],[0, 0]], requires_grad=True).send(alice)
target_alice = th.tensor([[0.],[0]], requires_grad=True).send(alice)

In [0]:
# create a linear model at local worker
model = nn.Linear(2,1)

In [0]:
# Send copies of the linear model to alice and bob
model_bob = model.copy().send(bob)
model_alice = model.copy().send(alice)

In [0]:
# create two opimizers for Alice and Bob
opt_bob = optim.SGD(params=model.parameters(), lr=0.1)
opt_alice = optim.SGD(params=model.parameters(), lr=0.1)

In [0]:

# train the models at alice and bob inside one loop
for i in range(10):

    # Train Bob's Model
    opt_bob.zero_grad()
    bobs_pred = model_bob(data_bob)
    bobs_loss = ((bobs_pred - target_bob)**2).sum()
    bobs_loss.backward()

    opt_bob.step()
    bobs_loss = bobs_loss.get().data

    # Train Alice's Model
    opt_alice.zero_grad()
    alices_pred = model_alice(data_alice)
    alices_loss = ((alices_pred - target_alice)**2).sum()
    alices_loss.backward()

    opt_alice.step()
    alices_loss = alices_loss.get().data
    alices_loss

In [0]:

# move the models to the third worker 
model_alice.move(secureWorker)
model_bob.move(secureWorker)
# aggreegate the models (their average)
# --- use model.weight.data to access the weights, and model.bias.data to access bias
# --- use model.weight.set_(new_weights) to update the weights
# --- use model.bias.set_(new_bias) to update the bias
with th.no_grad():

    model.weight.set_(((model_alice.weight.data + model_bob.weight.data) / 2).get())
    model.bias.set_(((model_alice.bias.data + model_bob.bias.data) / 2).get())

In [269]:
# make predictions uding the aggregated model
preds = model(data)
loss = ((preds - target) ** 2).sum()
print(preds)


tensor([[ 0.0890],
        [ 0.0704],
        [-0.4166],
        [-0.4351]], grad_fn=<AddmmBackward>)
